In [25]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))]))


cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468), # 평균 표준편차는 직접 구해서 입력 
                             (0.2470, 0.2435, 0.2616))
    ]))

label_map = {0: 0, 2: 1}
cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]

In [11]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
train_loader

In [13]:
import torch.nn as nn
import torch.optim as optim
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1)) 

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
n_epochs = 10

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.444639
Epoch: 1, Loss: 0.354587
Epoch: 2, Loss: 0.385355
Epoch: 3, Loss: 0.499176
Epoch: 4, Loss: 0.294243
Epoch: 5, Loss: 0.317107
Epoch: 6, Loss: 0.337321
Epoch: 7, Loss: 0.272299
Epoch: 8, Loss: 0.381347
Epoch: 9, Loss: 0.411768


In [42]:
import torch.nn as nn
import torch.optim as optim
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),) 

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 10

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.476721
Epoch: 1, Loss: 0.343208
Epoch: 2, Loss: 0.387167
Epoch: 3, Loss: 0.484015
Epoch: 4, Loss: 0.531947
Epoch: 5, Loss: 0.565086
Epoch: 6, Loss: 0.284170
Epoch: 7, Loss: 0.250047
Epoch: 8, Loss: 0.306394
Epoch: 9, Loss: 0.286066


In [30]:
for a,b in cifar2:
    a.shape

In [41]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        print('o',outputs)
        _, predicted = torch.max(outputs, dim=1)
        print('p',predicted)
        total += labels.shape[0]
        print('t',total)
        correct += int((predicted == labels).sum())
        print('c',correct)
   
print("Accuracy: %f" % (correct / total))

o tensor([[-3.7889e-01, -1.1540e+00],
        [-1.1797e-01, -2.1957e+00],
        [-6.7335e-02, -2.7315e+00],
        [-3.8690e+00, -2.1101e-02],
        [-6.7186e-01, -7.1490e-01],
        [-2.7873e+00, -6.3563e-02],
        [-1.6305e-01, -1.8941e+00],
        [-3.9099e+00, -2.0246e-02],
        [-3.1192e+00, -4.5198e-02],
        [-6.8074e-01, -7.0571e-01],
        [-5.3265e-01, -8.8442e-01],
        [-2.7678e-01, -1.4197e+00],
        [-5.2413e+00, -5.3075e-03],
        [-9.3517e-02, -2.4160e+00],
        [-1.0357e+00, -4.3847e-01],
        [-1.2910e-01, -2.1110e+00],
        [-1.4039e-01, -2.0327e+00],
        [-3.0668e-01, -1.3314e+00],
        [-1.1186e+00, -3.9562e-01],
        [-4.1518e+00, -1.5862e-02],
        [-1.4746e-01, -1.9870e+00],
        [-2.5841e+00, -7.8463e-02],
        [-4.0843e+00, -1.6979e-02],
        [-5.1174e-01, -9.1492e-01],
        [-6.3500e-02, -2.7883e+00],
        [-6.6625e-01, -7.2079e-01],
        [-3.5115e+00, -3.0306e-02],
        [-1.6000e+00, -2.2

In [37]:
labels

tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0])